In [ ]:
# importing basic packages

import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import plotly.express as px
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
from keras.models import load_model


In [ ]:
#given a series of predicted values, real values and holdout
#it compares the  n current days value to the n+holdout days value and assigns either 0 or 1
#1 if the value goes up 0 if it goes down
# creates a DF with both the real values and predicted values
def binarize(holdout,pred,test):
  test=test[:len(pred)]
  binaryTest = list()
  binaryPred = list()
  for i in range(0,len(pred)-holdout,holdout):
    if pred[i+holdout]>pred[i]:
      binaryPred.append(1)
    else:
      binaryPred.append(0)
    if test[i+holdout]>test[i]:
      binaryTest.append(1)
    else:
      binaryTest.append(0)
  return pd.DataFrame({"Real":binaryTest,"Pred":binaryPred})

In [ ]:
#returns the accuracy of your predictions based on the all the holdout values provided
def accuracyByHoldout(maxHoldout,pred,real):
  scores=list()
  for i in range(1,maxHoldout):
    data=binarize(i,pred,real)
    accuracy = accuracy_score(data["Real"], data["Pred"])

    scores.append(accuracy)
    #print("Holdout: ",i," Accuracy: ",accuracy)
  scoresDF= pd.DataFrame({"Holdout":range(1,maxHoldout),"Accuracy": scores})
  scoresDF=scoresDF.sort_values('Accuracy',ascending=False)
  return scoresDF


In [ ]:
#ored should be formated as numerical continous predictions
def returns(pred,test,holdout=1):
  test=test[:len(pred)]
  algoReturns = list()
  algoReturnsPercent = list()

  for i in range(0,len(pred)-holdout,holdout):
    nReturn = 0 #return for current trade
    if pred[i+holdout]>pred[i]:
      #go long
      nReturn = test[i+holdout]-test[i]
    else:
      #go short
      nReturn = test[i]-test[i+holdout]
    algoReturns.append(nReturn)

    #calculates the percent increase/decrease from day before
    algoReturnsPercent.append(nReturn/test[i])

  # returns the percent increase for each value
  # then the value increase for each trade
  # then the sum of the value increased for each trade
  return algoReturnsPercent,algoReturns,sum(algoReturns)


In [ ]:
#used to calculate the compounded value

#startMoney is default set to 1

#input should be the a list of the return percentages from the day before to current day
#start money
def compoundedReturns(AlgoReturnList,startMoney=1):
  CompRetAlgoList = list()
  CompRetAlgo = startMoney

  for i in range(0,int(len(AlgoReturnList))):
      CompRetAlgo = CompRetAlgo * (1+AlgoReturnList[i])

      CompRetAlgoList.append(CompRetAlgo)

  return CompRetAlgoList

In [1]:
def compReturnsByHoldout(pred,real,maxholdout):
  CompRetAlgoList = list()
  for i in range(1,maxholdout):
    returnList,_,_ = am.returns(pred,real,i)
    returnsList = list()
    for x in returnList:
      returnsList.append(x[0])
    compReturn=am.compoundedReturns(returnsList)

    CompRetAlgo = 1

    for i in range(0,int(len(returnsList))):
        CompRetAlgo = CompRetAlgo * (1+returnsList[i])

    CompRetAlgoList.append(CompRetAlgo)
  compReturnHoldout= pd.DataFrame({"Holdout":range(1,maxholdout),"Comp Returns": CompRetAlgoList})
  compReturnHoldout=compReturnHoldout.sort_values('Comp Returns',ascending=False)
  return compReturnHoldout.iloc[0,:],compReturnHoldout

SyntaxError: ignored

In [ ]:
def forecast(model,lookback,end_date= datetime.now()):

  #get the data from the past 'lookback+10' days just to make sure all the day has been recieved the only select last 'lookback' values
  start_date = end_date - timedelta(days=lookback+10)

  spyData = yf.download("SPY",start_date, end_date)
  print

  #selecting the vlose values and reshaping
  new_data = spyData.iloc[:,3:4]
  newCloseVals= new_data.Close.values
  newCloseVals= newCloseVals[-5:]
  #print(newCloseVals)
  newCloseVals = np.reshape(newCloseVals, (-1,1))


  # scaling dataset

  scaler = MinMaxScaler(feature_range=(0,1))
  scaled_close = scaler.fit_transform(newCloseVals)

  #adding the lagged values up the look back
  X_close = []
  for i in range(lookback, len(scaled_close)):
    X_close.append(scaled_close[i-lookback:i, 0])
  X_close = np.reshape(scaled_close, (1,-1))

  #print(X_close)
  #converting to numpy array
  X_close = np.array(X_close)

  #reshaping the array
  X_close = np.reshape(X_close, (X_close.shape[0], X_close.shape[1],1))
  #selecting only next day to predict
  #X_close=X_close[:1]

  #predict the next day value
  y_model = model.predict(X_close)
  #inverese the scaling
  y_model_O = scaler.inverse_transform(y_model)
  date = spyData[-1:].index+timedelta(days=1)

  return y_model_O[0][0], pd.DatetimeIndex(date)[0]

